## Explore parameter tuning with Randomized Search CV

In [1]:
from sklearn.model_selection import RandomizedSearchCV

model = RandomForestClassifier(n_jobs=-1, verbose=0)

print('Parameters currently in use:\n')
print(model.get_params())

NameError: name 'RandomForestClassifier' is not defined

In [3]:
import numpy

In [4]:
# Number of trees in random forest
n_estimators = [int(x) for x in numpy.linspace(start = 200, stop = 300, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in numpy.linspace(10, 100, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
#
max_leaf_nodes = [20, 25, 30, 35, 40]
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 211, 222, 233, 244, 255, 266, 277, 288, 300], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 19, 28, 37, 46, 55, 64, 73, 82, 91, 100, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
model_variables = ['red', 'nir', 'blue', 'green', 'swir1', 'swir2', 'sdev', 'edev']

model_col_indices = []

for model_var in model_variables:
    model_col_indices.append(column_names_indices[model_var])

In [6]:
model_rcv = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 80, cv = 3, verbose=2, random_state=42, n_jobs = -1)
classifier = model_rcv.fit(model_train[:,model_col_indices], model_train[:,0])

NameError: name 'model_cv' is not defined

In [ ]:
# classifier.best_params_
# {'n_estimators': 266,
#  'min_samples_split': 2,
#  'min_samples_leaf': 1,
#  'max_features': 'auto',
#  'max_depth': 37,
#  'bootstrap': False}

In [ ]:
base_model = RandomForestClassifier(n_estimators = 10, n_jobs=-1, verbose=0)
base_model.fit( model_train[:,model_col_indices], model_train[:,0])

In [ ]:
best_random = model_rcv.best_estimator_
random_accuracy = evaluate(best_random, model_test[:,model_col_indices], model_test[:,0])

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
# Variable importance
for var_name, var_importance in zip(model_variables, classifier.feature_importances_):
    print("{}: {:.04}".format(var_name, var_importance))

## Detailed parameter tuning with Grid Search CV

In [13]:
from sklearn.model_selection import GridSearchCV

In [37]:
param_grid = {
#     'bootstrap': [True],
    'max_depth': [25, 30, 35, 40, 45],
    'max_features': ['auto'],
    'min_samples_leaf': [1],
    'min_samples_split': [2, 3],
    'n_estimators': [300]
}

In [38]:
model_grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 3, verbose=2, n_jobs = -1)
model_grid.fit(model_train[:,model_col_indices], model_train[:,0])

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  4.7min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=-1,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='

In [39]:
print(model_grid.best_estimator_)
grid_accuracy = evaluate(model_grid.best_estimator_, model_test[:,model_col_indices], model_test[:,0])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=40, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
Model Performance
Average Error: 1.1830 degrees.
Accuracy = 99.38%.
